开始之前，让我们来看看 ElasticSearch，Kibana 和 Serilog 分别是什么。

什么是 ElasticSearch ？

简单来说，ElasticSearch 是一个开源数据库，非常适合索引日志和分析数据。

什么是 Kibana ？

Kibana 是开源的 ElasticSearch 的数据可视化用户界面。可以将 ElasticSearch 视为数据库，将Kibana 视为 Web 用户界面，您可以使用它在 ElasticSearch 中构建图表和查询数据。

什么是 Serilog ？

Serilog 是 ASP.NET Core 的一个插件，可以简化日志记录。Serilog 有各种可用的接收器，例如，有纯文本、SQL 和 ElasticSearch 接收器等等。

为什么 ElasticSearch 如此受欢迎？

除了几乎每个应用程序都需要日志记录这一事实之外，ElasticSearch 解决了许多问题并且做得非常好：

它是免费和开源的

免费。好吧，我承认是基本功能免费。如果您需要 Kibana 中的安全和警报功能，您可以购买 Kibana 的商业 X-pack 订阅，或者可以找一些开源的替代品。

RESTful API

ElasticSearch 有一个 RESTful 的 API。查询结果以 JSON 格式返回，这意味着结果是非常易用的。通过 RESTful API 查询和插入数据意味着使用任何编程语言都可以轻松使用 ElasticSearch。

易于查询

ElasticSearch 有一个内置的基于 Apache Lucene 的全文搜索引擎。与其他数据库相比，Lucene易于查询。即使是非技术人员也可以编写常见查询。

快 - 非常快

查询大型 SQL 数据库很容易花费 10 或 20 秒。对于大型 ElasticSearch 数据库上的类似查询，在 10 毫秒内返回结果是很常见的。

可扩展

它很容易扩展。再加上它是开源的, 这意味着您可以很容易控制您的钱包。

易于安装

只需启动包含 ElasticSearch 和 Kibana 容器的 docker compose 文件，您就可以开始记录和搜索了。

为什么我需要 ElasticSearch 和 Kibana ？

如果您曾经构建过应用程序，那么你一定记录过日志。我们经常去记录错误，但这些错误日志存储在服务器某处的文件文件里，经常会无法被访问。ElasticSearch 使任何类型的日志记录变得简单，易于访问和搜索。

ElasticSearch 令人难以置信的速度和简单的查询语言加上 Kibana 的图形界面，构成了强大的组合。如果您还没有使用 ElasticSearch 进行日志记录，我强烈建议您开始使用它。

信息足够了，让我们开始 Coding 。

我将使用 Visual Studio Code，一个开源的跨平台代码编辑器。出于本教程的目的，我将使用 Mac OSX，但您也可以使用 Ubuntu 或 Windows 10。

准备条件

要继续下面的步骤，请确保安装了这些：

Docker

Visual Studio Code

.NET Core SDK 2.1.300 或更高版本 ( 下载 )

创建项目文件夹

mkdir elastic-kibana

cd elastic-kibana

使用 .NET Core 命令行创建 MVC 项目

dotnet new mvc -n elastic-kibana -o src

在 Visual Studio Code 中打开项目

cd elastic-kibana

code .

创建 docker compose 文件

接下来，创建一个 docker compose 文件。此文件将一并启动 ElasticSearch 和 Kibana 容器，无需为每个容器运行单独的docker run命令。

mkdir docker

cd docker

创建一个名为 docker-compose.yml 的文件：

    version: '3.1'

    services:

      elasticsearch:
       image: docker.elastic.co/elasticsearch/elasticsearch:6.2.4
       container_name: elasticsearch
       ports:
        - "9200:9200"
       volumes:
        - elasticsearch-data:/usr/share/elasticsearch/data
       networks:
        - docker-network

      kibana:
       image: docker.elastic.co/kibana/kibana:6.2.4
       container_name: kibana
       ports:
        - "5601:5601"
       depends_on:
        - elasticsearch
       networks:
        - docker-network

    networks:
      docker-network:
        driver: bridge

    volumes:
      elasticsearch-data:
      
接下来，运行 docker compose 命令来启动容器。

国内优秀镜像源：

    Docker 官方中国区: https://registry.docker-cn.com
    网易: http://hub-mirror.c.163.com
    中科大: https://docker.mirrors.ustc.edu.cn

docker-compose up -d

第一次运行 docker-compose up 命令时，它将从 docker 仓库下载 ElasticSearch 和 Kibana 所需的镜像，因此它将花费几分钟的时间。

运行 docker-compose up 命令后，请检查 ElasticSearch 和 Kibana 是否已经启动。

ElasticSearch

打开 http://localhost:9200 确认 ElasticSearch 已经启动。

Kibana

打开 http://localhost:5601 确认 Kibana 已经启动。


加 Nuget 包到项目

我们将添加以下 Serilog 的包到项目。

Serilog

Serilog.Sinks.ElasticSearch

Serilog.Extensions.Logging

    cd ..
    cd elastic-kibana
    dotnet add package Serilog
    dotnet add package Serilog.Sinks.ElasticSearch
    dotnet add package Serilog.Extensions.Logging
    dotnet restore
    
添加一些配置信息到 appsettings.json

添加默认的日志设置以及 ElasticSearch 的 url 到 appsettings.json 文件

    {
      "Logging": {
        "LogLevel": {
            "Default": "Information",
            "System": "Information",
            "Microsoft": "Information"
        }
      },
      "ElasticConfiguration": {
        "Uri": "http://localhost:9200/"
      }
    }
    
在 Startup.cs 中配置日志功能

下一步，在 Startup.cs 中配置日志功能

添加这些 using 语句：

    using Microsoft.Extensions.Logging;
    using Serilog;
    using Serilog.Sinks.Elasticsearch;
    
然后，配置 Startup 的构造函数以从 appsettings.json 加载 ElasticSearch 的 URL，并配置 ElasticSearch 的接收器。

    public Startup(IConfiguration configuration, IHostingEnvironment hostingEnvironment)
    {
        var builder = new ConfigurationBuilder()
            .SetBasePath(hostingEnvironment.ContentRootPath)
            .AddJsonFile("appsettings.json", optional: true, reloadOnChange: true)
            .AddJsonFile($"appsettings.{hostingEnvironment.EnvironmentName}.json", reloadOnChange: true, optional: true)
            .AddEnvironmentVariables();

        Configuration = builder.Build();

        var elasticUri = Configuration["ElasticConfiguration:Uri"];

        Log.Logger = new LoggerConfiguration()
            .Enrich.FromLogContext()
            .WriteTo.Elasticsearch(new ElasticsearchSinkOptions(new Uri(elasticUri))
            {
                AutoRegisterTemplate = true,
            })
        .CreateLogger();
    }
    
最后，在 Configure 方法中将 Serilog 添加到日志工厂。

    public void Configure(IApplicationBuilder app, IHostingEnvironment env, ILoggerFactory loggerFactory)
    {        
        loggerFactory.AddSerilog();        
    }
    
开始记录日志到 ElasticSearch

现在，通过在 Visual Studio Code 中点击 F5 或者命令行中执行 dotnet run 来运行 MVC 应用程序。

启动 Kibana

由于我们在 Startup 类中配置了日志记录并将最小日志级别设置为 Information，因此运行该应用程序会将一些事件记录到 ElasticSearch 。

在 http://localhost:5601 打开 Kibana，以便我们可以查看日志。

加载 Kibana 后，您将看到默认页面。

在 Kibana 中创建索引模式来显示数据

Kibana 现在还不会显示任何日志。您必须先指定索引才能查看记录的数据。为此，请单击导航中的 Management 链接，然后将列在页面底部的 logstash 索引名称复制到文本框中，如下所示，然后单击下一步按钮。或者，您可以使用*号通配符，例如 logstash-*

然后，通过选择 @timestamp 指定时间过滤器字段名称，然后单击 Create index pattern按钮。

您现在可以通过单击导航中的 Discover 链接来查看日志。

在 MVC Controller 中记录自定义消息

由于我们指定要记录日志级别为 Information 或更高级别的消息，因此默认情况下会记录大量信息消息。但是如果我们想记录自己的消息呢？值得庆幸的是，这很容易做到。接下来我将在 HomeController 中记录一条消息。

添加 using 语句：

    using Microsoft.Extensions.Logging;
    
然后，使用构造函数注入的方式来注入 ILogger 的实例。

    ILogger<HomeController> _logger;
    public HomeController(ILogger<HomeController> logger)
    {
        _logger = logger;
    }
    
最后，在 Index Action 中记录一条消息。

    public IActionResult Index()
    {
        _logger.LogInformation($"oh hai there! : {DateTime.UtcNow}");

        return View();
    }
    
在 Kibana 中搜索

现在我们已经记录了一条消息，只需打开 Kibana 并搜索日志消息的文本即可。

您还可以将某一条日志在单独的窗口中打开，以查看各个字段记录的信息。

我将展示一些基本的搜索示例，来演示在 Kibana 中搜索的容易程度以及 ElasticSearch 的强大功能：

    message:"oh hai there"
    level:"Information"
    fields.ActionName:"elastic_kibana.Controllers.HomeController.Index"
    (message:"oh hai there" AND fields.ActionName:"elastic_kibana.Controllers.HomeController.Index")
    
记录错误日志到 ElasticSearch

一个典型的需求就是记录错误消息。使用 Serilog 这将变得非常简单，如下所示。

    try
    {
        throw new Exception("oops. i haz cause error in UR codez.");
    }
    catch (Exception ex)
    {
        _logger.LogError(ex, "ur code iz buggy.");
    }
    
在 Kibana 中搜索错误日志

在 Kibana 中找到错误日志非常简单，使用如下的搜索条件就可以找到所有的错误日志了。

    level: "Error"

我们来看看使用 Serilog 和 ElasticSearch 默认记录的详细信息。

它看起来还不错，但您会注意到异常细节被记录为一个大大的字符串。在此字符串中搜索信息仍会返回结果，但如果根据特定字段记录信息，我们可以执行更强大和特定的搜索。值得庆幸的是，有一个名为 Serilog.Exceptions 的插件可以帮助我们。

安装 Serilog.Exceptions Nuget 包:

    dotnet add package Serilog.Exceptions
    dotnet restore
    
接下来，在 Startup.cs 文件中使用如下 using 语句

    using Serilog.Exceptions;
    
然后，使用 Serilog.Exceptions 来丰富一下 Logger

    public Startup(IConfiguration configuration, IHostingEnvironment hostingEnvironment)
    {    
        Log.Logger = new LoggerConfiguration()
            .Enrich.FromLogContext()
            .Enrich.WithExceptionDetails()
            .WriteTo.Elasticsearch(new ElasticsearchSinkOptions(new Uri(elasticUri))
            {
                AutoRegisterTemplate = true,
            })
        .CreateLogger();
    }
    
最后，刷新一下，记录一个新的错误，并在 Kibana 中搜索到它，查看更结构化的错误日志记录。

降低日志级别

您可能会发现 Information 级别日志有点过于冗长，不符合您的口味。默认情况下，ASP.NET Core 将记录 Kestrel 托管相关的日志事件。这可能会非常嘈杂。排除掉它们的一种简单方法是通过修改 appsettings 文件，将 Microsoft 日志级别设置为 Warning 。

或者, 您可以通过将 Default 和 System 的最小日志级别设置为 Error 来进一步限制日志记录, 如下所示。

      "Logging": {
        "LogLevel": {
            "Default": "Error",
            "System": "Error",
            "Microsoft": "Warning"
        }
      }
      
尾声

传统的方法，日志记录需要大量的前期工作才能启动和运行。因此，日志记录通常会被完全遗漏，或者写入到难以访问的服务器上的某些模糊的文本文件里。

ElasticSearch 和 Kibana 改变了这一切。而 Docker 已经使 ElasticSearch 和 Kibana 的启动和运行变得毫不费力。ElasticSearch 和 Kibana 提供的强大功能以及非常高的性能，再加上它是开源的，这真的令人印象非常深刻。

即使没有像 Serilog 这样的插件，与传统 ASP.NET 相比，ASP.NET Core 中的日志记录也变得更加容易，因此, 在创建可扩展的日志记录框架方面, 我对 .NET Core 团队表示赞赏。Serilog 简单地构建在此之上，并且扩展了功能，以使 .NET Core 开发人员的日志记录变得更简单。

通过组合 Docker，ElasticSearch，Kibana，ASP.NET Core 和 Serilog ，您获得了前所未有的便利性和功能，再也没有理由不再将日志记录整合到应用程序中了。

下载源代码 : https://github.com/hrbhongwei/Elastic_Kibana

资源
    Download .NET Core 2.1 SDK
    ElasticSeach
    Kibana
    Serilog
    Serilog.Sinks.ElasticSearch Nuget Package

